In [46]:
import pandas as pd
from IPython.display import Image, HTML
from datetime import datetime

In [47]:
# convert link to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [48]:
dfs = pd.read_csv("inputs/dfs.csv")
fd = pd.read_csv("inputs/fanduel.csv")
dk = pd.read_csv("inputs/draftkings.csv")
st = pd.read_csv("inputs/statline.csv")
dfs = dfs.drop("Unnamed: 0", axis=1)
fd = fd.drop("Unnamed: 0", axis=1)
dk = dk.drop("Unnamed: 0", axis=1)
st = st.drop("Unnamed: 0", axis=1)

In [49]:
ts = datetime.now()
ts = ts.strftime("%b %d, %Y %I:%M %p")
ts = "Last Updated: " + ts

In [50]:
def highlight_dfs(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*19
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*19
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*19
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*19
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*19
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*19
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*19
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*19
    else: return ['background-color: white']*19

In [51]:
def highlight_fd(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*14
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*14
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*14
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*14
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*14
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*14
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*14
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*14
    else: return ['background-color: white']*14

In [52]:
def emojize(df):
    for i in range(len(df)):
        if df['Hot'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n🔥')
        if df['Hot'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Hot'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Cold'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n❄️')
        if df['Cold'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
        if df['Cold'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
    return df

In [53]:
def gen_page(df, out, coloring):
    games = pd.read_csv('games.csv').loc[:,['name','slug']]
    trans_games = games.set_index('name').T
    img_dict = trans_games.to_dict('list')
    images = []
    for i in df["Name"]:
        # placeholder pic for no slug
        code = "https://raw.githubusercontent.com/BerkeleySportsAnalytics/sagb-site/master/images/profile_picture.jpg"
        # Donta Hall and Paul Watson do not have profile pictures, as of 8/7/20
        if img_dict.get(i) != None and (i != "Paul Watson" and i != "Donta Hall"):
            code = "https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/" + img_dict.get(i)[0] + ".jpg"
        images.append(code)
    df["Picture"] = images
    
    cols = df.columns.tolist()
    cols = [cols[0]] + cols[-1:] + cols[1:-1]
    df = df[cols]
    pd.set_option('display.max_colwidth', -1)
    df.index +=1
    pre = (
        "<head>\n<script src=\"sorttable.js\"></script>\n"
        "<link rel=\"stylesheet\" href=\"df_styles.css\">\n</head>\n"
    )
    pre = pre + ts + "\n"
    pre += (
        "\n<body>\n<div id=\"filter\">\n<input type=\"text\" id=\"name\" onkeyup=\"nameSearch()\" placeholder=\"Search for name...\">\n"
        "<input type=\"text\" id=\"team\" onkeyup=\"teamSearch()\" placeholder=\"Search for team...\">\n"
        "<label for=\"pos\">Search by position:</label>\n"
        "<select id=\"pos\" onchange=\"posSearch()\" class='form-control'>\n"
        "<option></option>\n"
        "<option>PG</option>\n"
        "<option>SG</option>\n"
        "<option>SF</option>\n"
        "<option>PF</option>\n"
        "<option>C</option>\n"
        "</select>\n"
        "</div>\n\n"
    )
    
    post = (
        "\n"
        "\n<script src=\"main.js\"></script>"
        "\n</body>"
        )
    # render dataframe as html
    if coloring is not None:
        images = ['<img src="' + image + '" width="60">' for image in images]
        df["Picture"] = images
        df = emojize(df)
        body = df.style.apply(coloring, axis=1).hide_columns(['Hot', 'Cold']).render()
    else:
        html = HTML(df.to_html(escape=False ,formatters=dict(Picture=path_to_image_html)))
        body = html.data

    body = body.replace("<table id=", "<table ignore=", 1)
    body = body.replace("<table ", "<table id=\"tbl\" class=\"sortable\" ", 1)
    a = pre + body + post
    # write html to file
    with open(out, 'w', encoding="utf-8") as f:
        f.write(a)
    return df

In [54]:
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(fd, 'fd.html', highlight_fd)
gen_page(dk, 'dk.html', highlight_fd)
gen_page(st, 'st.html', None)

C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/

,Name,Picture,Position,Game,Team,Minutes,2PT FG,3PT FG,FTM,Rebounds,Assists,Blocks,Steals,Turnovers,Opponent Defensive Rank vs Position,Injury Indicator,Injury Details
1,Joel Embiid,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/embiijo01.jpg,C,ORL@PHI,PHI,35.56,5.62,0.81,8.55,9.94,1.70,1.54,0.55,3.19,7.0,,
2,De'Aaron Fox,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/foxde01.jpg,PG,SAC@BKN,SAC,35.60,7.01,1.34,5.93,2.40,3.31,0.17,1.02,3.04,20.0,,
3,Ben Simmons,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/simmobe01.jpg,PG,ORL@PHI,PHI,35.80,4.09,0.60,4.28,8.56,6.97,0.56,1.57,3.29,14.0,O,Kneecap
4,Caris LeVert,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/leverca01.jpg,SG,SAC@BKN,BKN,32.44,5.60,1.70,5.04,4.35,4.14,0.38,0.89,3.07,23.0,,
5,Chris Paul,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/paulch01.jpg,PG,OKC@MEM,OKC,30.24,4.04,1.68,3.69,3.32,6.80,0.06,1.27,2.56,22.0,,
6,Nikola Vucevic,https://raw.githubusercontent.com/BerkeleySportsAnalytics/sagb-site/master/images/profile_picture.jpg,C,ORL@PHI,ORL,26.01,5.50,0.84,1.94,8.49,4.23,0.61,0.65,1.01,15.0,,
7,Ja Morant,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/moranja01.jpg,PG,OKC@MEM,MEM,28.37,5.22,1.36,3.70,3.30,5.93,0.53,1.14,3.50,17.0,,
8,Jrue Holiday,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/holidjr01.jpg,SG,WAS@NO,NO,34.85,4.32,2.19,2.65,5.20,5.96,0.23,1.38,3.36,18.0,,
9,Thomas Bryant,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/bryanth01.jpg,C,WAS@NO,WAS,20.03,1.75,0.58,0.98,4.93,1.24,0.44,0.52,0.89,14.0,,
10,Brandon Ingram,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/ingrabr01.jpg,SF,WAS@NO,NO,14.63,3.66,1.65,2.80,4.98,2.38,0.52,0.64,2.09,11.0,,
